In [57]:
from html.parser import HTMLParser
import re
converted_text = ''
img_found = False
anchor_found = False
code_found = False


# code for preprocessing

def rem_digit(str):
    str_without_digit = str.translate({ord(ch): None for ch in '0123456789.'})
    return str_without_digit


class MyHTMLParser(HTMLParser):
    
    def handle_starttag(self, tag, attrs):
        global converted_text, img_found, anchor_found, code_found
        #print("starting tag...", tag)
        if 'code' in tag:
            code_found = True
        if tag == 'img':
            converted_text += '<IMG> '
        if tag == 'a':
            converted_text += '<ANCHOR> '
        #print("code found start !!! ", code_found)   
    def handle_endtag(self, tag):
        global converted_text, img_found, anchor_found, code_found
        #print("ending tag...", tag)
        if tag == 'code':
            code_found = False
    def handle_data(self, data):
        #print("data...", data)
        global converted_text, img_found, anchor_found, code_found
        str_without_digit = ' ' +  re.sub(r'\b\d+(?:\.\d+)?\s+', '', data) # remove digits
        final_data = ''.join(str_without_digit.split("\n")) # remove newlines
        final_data = re.sub(r'http\S+', '', final_data, flags=re.MULTILINE)
        if code_found:
                converted_text += '<CODE>' + final_data + '<CODE>'
        else:  
            converted_text += final_data
        #print("converted text....", converted_text)    

        
parser = MyHTMLParser()




In [58]:
import gensim, logging
import nltk, csv
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup

train_file = './sample_train_1000.csv'
tags_file = './tags_1000.json'
word2vec_model = 'word2vec_1000_new.model'


class MySentences(object):
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        stop_words = set(stopwords.words('english'))
        global converted_text
        with open(self.filename) as csvfile:
            reader = csv.reader(csvfile)
            for rowIndex, row in enumerate(reader):
                if 0 < rowIndex:
#                     soup = BeautifulSoup(row[2], 'lxml')
#                     body_data = []
#                     for i, content in enumerate(soup.find_all('p')):
#                         body_data.append(content.string)
#                     body = body_data[:200]
#                     body_data_1 = ''.join(str(e) + " " for e in body)
#                     body_data_1 = body_data_1.lower()
#                     title_tag = (row[1] + " " + body_data_1 + " ").split(" ")
#                     final_words = []
#                     for word in title_tag:
#                         if word not in stop_words:
#                             final_words.append(word)
#                     yield final_words

                        for index, item in enumerate(row):
                            if index == 2:
                                parser.feed(item)
                                fin_text = rem_digit(converted_text)
                        converted_text = ''
                        img_found = False
                        anchor_found = False
                        code_found = False
                        fin_text = (row[1] + " " + fin_text + " " + row[3]).lower()
                        final_words = []
                        for word in fin_text.split(" "):
                            if word not in stop_words:
                                final_words.append(word)
                        yield final_words
                                

def create_word2vec_model():
    sentences = []
    sentences = MySentences(train_file)  # a memory-friendly iterator
    gensim_model_1 = gensim.models.Word2Vec(sentences, min_count=1, size=150, window=10, workers=10)
    
    print(gensim_model_1.wv.most_similar(positive='java', topn=6))
    # print(gensim_model_1.wv.most_similar(positive='math', topn=6))
    print(gensim_model_1.wv.most_similar(positive='html', topn=6))
    print(gensim_model_1.wv.most_similar(positive='android', topn=6))
    print(gensim_model_1.wv.most_similar(positive='php', topn=6))
    gensim_model_1.save(word2vec_model)
    print("model created")    

create_word2vec_model()  


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('run', 0.9999842643737793), ('want', 0.9999836683273315), ('able', 0.9999834895133972), ('every', 0.9999769926071167), ('display', 0.999976634979248), ('since', 0.9999759793281555)]
[('image', 0.9999802112579346), ('create', 0.9999775886535645), ('point', 0.9999713897705078), ('main', 0.9999713897705078), ('used', 0.9999712109565735), ('call', 0.999961793422699)]
[('files', 0.9999815225601196), ('created', 0.9999809265136719), ('client', 0.9999803304672241), ('show', 0.9999784827232361), ('update', 0.9999756813049316), ('number', 0.9999730587005615)]
[('order', 0.9999838471412659), ('code', 0.9999830722808838), ('thanks', 0.9999815821647644), ('interface', 0.9999757409095764), ('delete', 0.9999692440032959), ('css', 0.9999679327011108)]
model created


In [59]:
# we are calculating second highest because we are taking cosine similarity over entire vector space. 
# the first highest will always be 1 because a(dot)a = 1

def get_second_highest(mat):
    global_max_x = 0
    global_max_y = 0
    global_max = -1000
    for rowIndex, row in enumerate(mat):
        a = row
        hi = mid = -1000
        for index, x in enumerate(a):
            if x > hi:
                mid = hi
                hi = x
            elif x < hi and x > mid:
                lo = mid
                mid = x
        try:        
            index_max_2 = list(a).index(mid)
            if mid > global_max:
                global_max_x = rowIndex
                global_max_y = index_max_2
                global_max = mid
        except:
            pass
    return global_max_x, global_max_y



In [61]:
# initially all node are going to be cluster each
import networkx as nx
import numpy as np, json
from sklearn.metrics.pairwise import cosine_similarity as cosine
import pickle

clusters = []
global_node_counter = 0
H = nx.DiGraph()
graph_pickle = "HA_g_1000_new.p"



model = gensim.models.Word2Vec.load(word2vec_model)
model_words = model.wv


with open(tags_file) as f:
    data = json.load(f)
    tag_set = data['tags']


def avg_centroid(centroid_1, centroid_2):
    new_centroid = np.zeros((len(centroid_1), ), dtype=float)
    for i in range(len(centroid_1)):
        val = (centroid_1[i] + centroid_2[i]) / 2
        new_centroid[i] = val
    return new_centroid

# clusters will eventually perish
# Graph would hold all necessary information
for key in tag_set:
    if key in model_words:
        cluster = {}
        cluster['tag'] = True
        cluster['word'] = [key]
        cluster['centroid'] = model[key]
        cluster['id'] = global_node_counter
        cluster['root'] = False
        clusters.append(cluster)
        global_node_counter += 1
        H.add_node(cluster['id'], tag=cluster['tag'],word=cluster['word'], centroid=cluster['centroid'], root=cluster['root'])
        
print("graph created")

while len(clusters) > 1:
    simi_mat_1 = np.zeros(shape=(len(clusters), 150))
    print(np.shape(simi_mat_1))
    for rowIndex, row in enumerate(simi_mat_1):
        simi_mat_1[rowIndex] = clusters[rowIndex]['centroid']
    result = cosine(simi_mat_1, simi_mat_1)
    second_best_index_x, second_best_index_y = get_second_highest(result) # get the index of second best
    cluster_x, cluster_y = clusters[second_best_index_x], clusters[second_best_index_y]
    
    # creating new cluster
    new_cluster = {}
    new_cluster['tag'] = False
    new_cluster['word'] = cluster_x['word'] + cluster_y['word']
    new_cluster['centroid'] = avg_centroid(cluster_x['centroid'], cluster_y['centroid'])
    new_cluster['id'] = global_node_counter
    new_cluster['root'] = True if len(clusters) == 2 else False 
    global_node_counter += 1
    try:
        clusters.remove(cluster_x)
        clusters.remove(cluster_y)
    except:
        print("ERROR............", second_best_index_y, len(clusters))
        print("cluster x ", cluster_x)
        print("cluster y ", cluster_y)
        
        break
    clusters.append(new_cluster)
    H.add_node(new_cluster['id'], tag=new_cluster['tag'],word=new_cluster['word'], centroid=new_cluster['centroid'], root=new_cluster['root'])
    H.add_edge(new_cluster['id'], cluster_x['id'])
    H.add_edge(new_cluster['id'], cluster_y['id'])
    print("len clusters ", len(clusters))
    
    
pickle.dump(H, open("HA_g_1000_new.p", "wb"))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


graph created
(1364, 150)
len clusters  1363
(1363, 150)
len clusters  1362
(1362, 150)
len clusters  1361
(1361, 150)
len clusters  1360
(1360, 150)
len clusters  1359
(1359, 150)
len clusters  1358
(1358, 150)
len clusters  1357
(1357, 150)
len clusters  1356
(1356, 150)
len clusters  1355
(1355, 150)
len clusters  1354
(1354, 150)
len clusters  1353
(1353, 150)
len clusters  1352
(1352, 150)
len clusters  1351
(1351, 150)
len clusters  1350
(1350, 150)
len clusters  1349
(1349, 150)
len clusters  1348
(1348, 150)
len clusters  1347
(1347, 150)
len clusters  1346
(1346, 150)
len clusters  1345
(1345, 150)
len clusters  1344
(1344, 150)
len clusters  1343
(1343, 150)
len clusters  1342
(1342, 150)
len clusters  1341
(1341, 150)
len clusters  1340
(1340, 150)
len clusters  1339
(1339, 150)
len clusters  1338
(1338, 150)
len clusters  1337
(1337, 150)
len clusters  1336
(1336, 150)
len clusters  1335
(1335, 150)
len clusters  1334
(1334, 150)
len clusters  1333
(1333, 150)
len clusters 

len clusters  1099
(1099, 150)
len clusters  1098
(1098, 150)
len clusters  1097
(1097, 150)
len clusters  1096
(1096, 150)
len clusters  1095
(1095, 150)
len clusters  1094
(1094, 150)
len clusters  1093
(1093, 150)
len clusters  1092
(1092, 150)
len clusters  1091
(1091, 150)
len clusters  1090
(1090, 150)
len clusters  1089
(1089, 150)
len clusters  1088
(1088, 150)
len clusters  1087
(1087, 150)
len clusters  1086
(1086, 150)
len clusters  1085
(1085, 150)
len clusters  1084
(1084, 150)
len clusters  1083
(1083, 150)
len clusters  1082
(1082, 150)
len clusters  1081
(1081, 150)
len clusters  1080
(1080, 150)
len clusters  1079
(1079, 150)
len clusters  1078
(1078, 150)
len clusters  1077
(1077, 150)
len clusters  1076
(1076, 150)
len clusters  1075
(1075, 150)
len clusters  1074
(1074, 150)
len clusters  1073
(1073, 150)
len clusters  1072
(1072, 150)
len clusters  1071
(1071, 150)
len clusters  1070
(1070, 150)
len clusters  1069
(1069, 150)
len clusters  1068
(1068, 150)
len clus

len clusters  823
(823, 150)
len clusters  822
(822, 150)
len clusters  821
(821, 150)
len clusters  820
(820, 150)
len clusters  819
(819, 150)
len clusters  818
(818, 150)
len clusters  817
(817, 150)
len clusters  816
(816, 150)
len clusters  815
(815, 150)
len clusters  814
(814, 150)
len clusters  813
(813, 150)
len clusters  812
(812, 150)
len clusters  811
(811, 150)
len clusters  810
(810, 150)
len clusters  809
(809, 150)
len clusters  808
(808, 150)
len clusters  807
(807, 150)
len clusters  806
(806, 150)
len clusters  805
(805, 150)
len clusters  804
(804, 150)
len clusters  803
(803, 150)
len clusters  802
(802, 150)
len clusters  801
(801, 150)
len clusters  800
(800, 150)
len clusters  799
(799, 150)
len clusters  798
(798, 150)
len clusters  797
(797, 150)
len clusters  796
(796, 150)
len clusters  795
(795, 150)
len clusters  794
(794, 150)
len clusters  793
(793, 150)
len clusters  792
(792, 150)
len clusters  791
(791, 150)
len clusters  790
(790, 150)
len clusters  

len clusters  538
(538, 150)
len clusters  537
(537, 150)
len clusters  536
(536, 150)
len clusters  535
(535, 150)
len clusters  534
(534, 150)
len clusters  533
(533, 150)
len clusters  532
(532, 150)
len clusters  531
(531, 150)
len clusters  530
(530, 150)
len clusters  529
(529, 150)
len clusters  528
(528, 150)
len clusters  527
(527, 150)
len clusters  526
(526, 150)
len clusters  525
(525, 150)
len clusters  524
(524, 150)
len clusters  523
(523, 150)
len clusters  522
(522, 150)
len clusters  521
(521, 150)
len clusters  520
(520, 150)
len clusters  519
(519, 150)
len clusters  518
(518, 150)
len clusters  517
(517, 150)
len clusters  516
(516, 150)
len clusters  515
(515, 150)
len clusters  514
(514, 150)
len clusters  513
(513, 150)
len clusters  512
(512, 150)
len clusters  511
(511, 150)
len clusters  510
(510, 150)
len clusters  509
(509, 150)
len clusters  508
(508, 150)
len clusters  507
(507, 150)
len clusters  506
(506, 150)
len clusters  505
(505, 150)
len clusters  

(247, 150)
len clusters  246
(246, 150)
len clusters  245
(245, 150)
len clusters  244
(244, 150)
len clusters  243
(243, 150)
len clusters  242
(242, 150)
len clusters  241
(241, 150)
len clusters  240
(240, 150)
len clusters  239
(239, 150)
len clusters  238
(238, 150)
len clusters  237
(237, 150)
len clusters  236
(236, 150)
len clusters  235
(235, 150)
len clusters  234
(234, 150)
len clusters  233
(233, 150)
len clusters  232
(232, 150)
len clusters  231
(231, 150)
len clusters  230
(230, 150)
len clusters  229
(229, 150)
len clusters  228
(228, 150)
len clusters  227
(227, 150)
len clusters  226
(226, 150)
len clusters  225
(225, 150)
len clusters  224
(224, 150)
len clusters  223
(223, 150)
len clusters  222
(222, 150)
len clusters  221
(221, 150)
len clusters  220
(220, 150)
len clusters  219
(219, 150)
len clusters  218
(218, 150)
len clusters  217
(217, 150)
len clusters  216
(216, 150)
len clusters  215
(215, 150)
len clusters  214
(214, 150)
len clusters  213
(213, 150)
len

In [124]:
import pickle
from sklearn.metrics.pairwise import cosine_similarity as cosine
import warnings
test_file = './sample_test_100.csv'
warnings.filterwarnings('ignore')



def preprocess(content):
    soup = BeautifulSoup(content, 'lxml')
    body_data = []
    final_words = []


    for i, content in enumerate(soup.find_all('p')):
        body_data.append(content.string)
        body = body_data[:200]
        body_data_1 = ''.join(str(e) + " " for e in body)
        body_data_1 = body_data_1.lower()
        title_tag = body_data_1.split(" ")
        for word in title_tag:
            if word not in stop_words:
                final_words.append(word)

    return final_words


graph_pickle = "HA_g_1000_new.p"
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load(word2vec_model, mmap='r')
graph = pickle.load(open(graph_pickle, "rb"))
for node in list(graph.nodes(data='root')): 
    id, prop = node
    if prop:
        root_node = id
        break
print("root node")
print(graph.node[root_node]['centroid'])
print("length ", len(graph.node[root_node]['word']))
# print(graph.node[root_node]['word'])
accuracy = 0


with open(test_file, 'r') as csvfile:
    reader = csv.reader(csvfile)
    for rowIndex, row in enumerate(reader):
        if rowIndex == 3 :
            # print(row[2])
            path = []
            test_embedding_1 = np.zeros(150)
            test_embedding_2 = np.zeros(150)

            
            count1 = 0
            count2 = 0
            error_count = 0
            title_content = preprocess(row[1])
            for word in title_content:
                if word in word_vectors and not word.isspace():
                    count1 += 1
                    test_embedding_1 = np.add(test_embedding_1, word_vectors[word])
            if np.count_nonzero(test_embedding_1) > 0:
                test_embedding_1 = test_embedding_1 / count1  

            body_content = preprocess(row[2])
            for word in body_content:
                if word in word_vectors and not word.isspace():
                    count2 += 1
                    # print("word ", word)
                    test_embedding_2 = np.add(test_embedding_2, word_vectors[word])
            if np.count_nonzero(test_embedding_2) > 0:
                test_embedding_2 = test_embedding_2 / count2
            if count1 == 0 and count2 == 0:
                error_count += 1
                continue
            test_embedding = np.zeros(150)
            test_embedding = np.add(test_embedding, test_embedding_1, test_embedding_2)
            # print("test embedding ", test_embedding)
            current_node = root_node
            path.append(current_node)
            while len(list(graph.neighbors(current_node))) != 0:
                similarity = -1000
                next_node = -1
                neighbors = list(graph.neighbors(current_node))
                # print("neighbors of current_node ", current_node)
                # print("neighbors ", neighbors)
                siblings = []
                for node in neighbors:
                    siblings.append(node)
                    cent = graph.node[node]['centroid']
                    try:
                        cos_sim = cosine([test_embedding], [cent])
                        print("cos sime ", node, cos_sim)
                        if cos_sim > similarity:
                            next_node = node
                            similarity = cos_sim
                    except:
                        print(" error....in embedding ", rowIndex)
                        # print(test_embedding)
                    # print(" node cos sim ", node, cos_sim)
                #print("next node ", next_node)
                #print("simil ", similarity)
                print("picked node", next_node)
                siblings.remove(next_node)
                current_node = next_node
                path.append(current_node)
            print(" solution ", graph.node[current_node]['word'])  
            print(" solution ", graph.node[current_node]['tag'])  


            gold_labels = row[3].split(" ")
            print(gold_labels)
            print(path)
            if graph.node[current_node]['word'][0] in gold_labels:
                accuracy += 1
#         if rowIndex == 10:
#             break
print("accuracy ", accuracy)            
print("ERROR COUNT ", error_count)
print("sib len ", len(siblings), siblings[0])
print(graph.node[siblings[0]]['word'])
print(graph.node[siblings[0]]['tag'])


# for n,d in graph.nodes_iter(data=True):
#     print(d)
#     break
# print(graph.node[303])
# print(list(graph.predecessors(0)))
# for item in list(graph.predecessors(0)):
#     print(item)


root node
[-4.51514159e-04 -6.99703780e-04  1.57199120e-03 -1.35576933e-03
 -5.86048136e-04  4.71978127e-04 -6.69234109e-04  1.35340851e-03
  7.74051872e-04 -9.60690125e-04 -6.16129718e-04  5.68162035e-04
 -8.14203777e-04  8.17934553e-04 -1.17089809e-03  9.87746324e-04
 -1.20892435e-03  6.95985810e-05  5.38964400e-04 -6.71568614e-04
  1.65539160e-03  6.90740320e-04 -1.28465497e-03 -3.16092604e-05
  5.14318305e-04  3.76577621e-04 -4.54794016e-04 -9.98333883e-05
  2.02687049e-04 -5.68116297e-04  7.19206051e-04 -9.29231602e-04
 -1.91444218e-04 -2.56334726e-04  2.65264454e-04 -2.76430175e-04
 -5.26711662e-04  1.41927737e-03  1.66335229e-04 -3.51040884e-04
  1.05532704e-04  3.05365110e-04 -4.51571265e-04  4.74691550e-04
 -9.16904084e-04 -3.54992989e-04  4.11163989e-04  4.71593739e-04
  1.05493743e-03  5.17650672e-05 -2.19881131e-04 -9.09658919e-04
  3.89307538e-04  1.10000015e-03  5.93480167e-04 -8.60237126e-04
  1.05449035e-03 -7.61731445e-04 -8.61307377e-04 -6.78522494e-04
 -1.07374894e-0

In [63]:
def recall(y_pred, y_true):
    i = set(y_true).intersection(y_pred)
    return len(i) / len(y_true)


def precision(y_pred, y_true):
    i = set(y_true).intersection(y_pred)
    len_all = len(y_pred)
    if len_all == 0:
        return 0
    else:
        return len(i) / len_all


def f1(y_pred, y_true):
    p = precision(y_pred, y_true)
    r = recall(y_pred, y_true)
    if p + r == 0:
        return 0
    else:
        return 2 * (p * r) / (p + r)

In [64]:
f_score = 0
with open(test_file, 'r') as csvfile:
    reader = csv.reader(csvfile)
    for rowIndex, row in enumerate(reader):
        if 0 < rowIndex:
            test_embedding = np.zeros(150)
            result = []
            print(row[1])
            for word in row[1].split(" "):
                if word in tag_set and word not in result:
                    result.append(word)
            for word in row[2].split(" "):
                if word in tag_set and word not in result:
                    result.append(word)
            gold_labels = row[3].split(" ")
            print("xx ", result)
            print(gold_labels)
            f_score += f1(result, gold_labels)
            if rowIndex == 1:
                break
print(f_score)            

How to check if an uploaded file is an image without mime type?
xx  ['file', 'image', 'like', 'upload']
['php', 'image-processing', 'file-upload', 'upload', 'mime-types']
0.22222222222222224


In [65]:
print(len(graph.nodes))

2727


In [66]:
test_embedding = word_vectors['php']
current_node = root_node
path.append(current_node)
while len(list(graph.neighbors(current_node))) != 0:
    similarity = -1000
    next_node = -1
    neighbors = list(graph.neighbors(current_node))
    for node in neighbors:
        cent = graph.node[node]['centroid']
        cos_sim = cosine([test_embedding], [cent])
                    # print(" node cos sim ", node, cos_sim)
        if cos_sim > similarity:
            next_node = node
            similarity = cos_sim
                #print("next node ", next_node)
                #print("simil ", similarity)
            current_node = next_node
            path.append(current_node)

print(graph.node[current_node]['word'][0])
print(word_vectors.wv.most_similar(positive='java', topn=6))
print(word_vectors.wv.most_similar(positive='math', topn=6))
print(word_vectors.wv.most_similar(positive='html', topn=6))
print(word_vectors.wv.most_similar(positive='android', topn=6))
print(word_vectors.wv.most_similar(positive='php', topn=6))



axiom-of-choice
[('run', 0.9999842643737793), ('want', 0.9999836683273315), ('able', 0.9999834895133972), ('every', 0.9999769926071167), ('display', 0.999976634979248), ('since', 0.9999759793281555)]
[('type-cast', 0.8781887292861938), ('allowfullscreen', 0.8763707280158997), ('concerns,', 0.875644326210022), ('constraints', 0.8755163550376892), ('effects', 0.8754645586013794), ('"+"the', 0.8750550746917725)]
[('image', 0.9999802112579346), ('create', 0.9999775886535645), ('point', 0.9999713897705078), ('main', 0.9999713897705078), ('used', 0.9999712109565735), ('call', 0.999961793422699)]
[('files', 0.9999815225601196), ('created', 0.9999809265136719), ('client', 0.9999803304672241), ('show', 0.9999784827232361), ('update', 0.9999756813049316), ('number', 0.9999730587005615)]
[('order', 0.9999838471412659), ('code', 0.9999830722808838), ('thanks', 0.9999815821647644), ('interface', 0.9999757409095764), ('delete', 0.9999692440032959), ('css', 0.9999679327011108)]
